In [1]:
%load_ext autoreload
%autoreload 2

import os 
import shutil
import numpy as n
from datetime import date
from matplotlib import pyplot as plt
import time
import napari

In [2]:
os.chdir('/home/ali/packages/s2p-lbm/')

from suite3d.job import Job
from suite3d import lbmio, utils, ui
from suite3d import tiff_utils as tfu
from suite3d import file_utils as flu

/home/ali/anaconda3/envs/suite3d-gpu/lib/python3.8/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/home/ali/anaconda3/envs/suite3d-gpu/lib/python3.8/site-packages/paramiko/transport.py:258: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [27]:
subjects_dir =   '/mnt/zortex-subjects/'
expt_info = { 
    'subject':         'CR031',
    'date' :          '2024-12-06',
    'expnum' :         [1],}

tifs, si_params, exp_str = flu.find_exp(subjects_dir, **expt_info, verbose=False)

job_params = {    
    # number of channels recorded in the tiff file, typically 30
    'n_ch_tif' : 26,
    # number of planes in the deeper cavity, typically 15
    'cavity_size' : 13,
    # convert from the Scanimage channel ordering to deep-to-shallow ordering
    # in our case, ScanImage channel numbers are in temporal order. 
    'planes' : n.array([ 0,  2,  4,  6,  8, 10, 12, 14,
                        16, 18, 20, 22]),
    # number of files to use for the initial pass
    'n_init_files' :   4,
    
    # number of pixels to fuse between the ROI strips
    # the auto-detection doesn't always work well, recommend 
    # manually tuning it to reduce the stitching artifacts
    'fuse_shift_override' : 6,
    
    # will try to automatically estimate crosstalk using 
    # the shallowest crosstalk_n_planes planes. if you want to override,
    # set override_crosstalk = float between 0 and 1
    'subtract_crosstalk' : False,
    
    # volume rate in acquisition
    'fs' : flu.get_si_params(tifs[0])['vol_rate'],
    
    # 3D GPU registration - fast! 
    # for now, 3D + GPU are well-tested, other options might have minor bugs
    # if you run into them, let us know! we'll fix them
    '3d_reg' : True,
    'gpu_reg' : True,
}


In [28]:

job_params['fs'] = si_params['vol_rate']

savedir = os.path.join('/mnt/md0/runs', expt_info['subject'], expt_info['date'])
os.makedirs(savedir, exist_ok=True)

# Create the job
job = Job('/mnt/md0/runs',exp_str, tifs = tifs,
          params=job_params, create=True, overwrite=True, verbosity = 3)

Loading job directory for CR031_2024-12-06_1 in /mnt/md0/runs
      Created dir /mnt/md0/runs/s3d-CR031_2024-12-06_1/registered_fused_data
      Created dir /mnt/md0/runs/s3d-CR031_2024-12-06_1/summary
      Created dir /mnt/md0/runs/s3d-CR031_2024-12-06_1/iters
   Loading default params
      Updating param n_ch_tif
      Updating param cavity_size
      Updating param planes
      Updating param n_init_files
      Updating param fuse_shift_override
      Updating param subtract_crosstalk
      Updating param fs
      Updating param 3d_reg
      Updating param gpu_reg
   Updated main params file


In [29]:
# optional parameters for initialization
# load 1 file to initialize, and select 200 frames randomly to do initalization on
job.params['n_init_files'] = 4
job.params['init_n_frames'] = None
job.params['init_file_sample_method'] = 'even'
job.params['voxel_size_um'] = (20, 2.5, 2.5)
job.params['fix_shallow_plane_shift_estimates'] = False

In [42]:
# %%time
# summary = job.run_init_pass()

In [ ]:
summary = job.load_summary()
img = summary['ref_img_3d']

In [43]:
nbui.ImageSliceViewer3D(img)

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [44]:
# split the large tiffs into files of size 100 after registration
job.params['split_tif_size'] = 100

In [45]:
%%time
job.register()

      Found dir /mnt/md0/runs/s3d-CR031_2024-12-06_1/registered_fused_data
      Updating self.dirs tag registered_fused_data
   Saved a copy of params at /mnt/md0/runs/s3d-CR031_2024-12-06_1/registered_fused_data
   Updated main params file
   Starting registration: 3D: True, GPU: True
   Overriding fuse shift value to 6
   Not clipping frames for registration
Will analyze 55 tifs in 55 batches
   Enforcing positivity
   Launching IO thread
         Memory at batch 0.  Total Used: 129.358 GB, Virtual Available: 376.754 GB, Virtual Used: 126.442 GB, Swap Used: 002.916 GB
Loading Batch 0 of 54
      Loading /mnt/zortex-subjects/CR031/2024-12-06/1/2024-12-06_1_CR031_2P_00001_00001.tif
    Loaded file into shared memory in 1.84 sec


Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid st

    Workers completed in 0.98 sec
    Total time: 2.82 sec
   Loaded 1 files, total 0.32 GB
   Batch 0 IO thread joined
         Memory after IO thread joinTotal Used: 129.704 GB, Virtual Available: 376.408 GB, Virtual Used: 126.788 GB, Swap Used: 002.916 GB
         Memory after movie copied from threadTotal Used: 130.020 GB, Virtual Available: 376.091 GB, Virtual Used: 127.105 GB, Swap Used: 002.916 GB
         Memory after thread memory clearedTotal Used: 129.699 GB, Virtual Available: 376.412 GB, Virtual Used: 126.783 GB, Swap Used: 002.916 GB
   Launching IO thread for next batch
         After IO thread launch:Total Used: 129.699 GB, Virtual Available: 376.412 GB, Virtual Used: 126.783 GB, Swap Used: 002.916 GB
      Loaded batch of size (12, 50, 500, 576)
      Loading /mnt/zortex-subjects/CR031/2024-12-06/1/2024-12-06_1_CR031_2P_00001_00002.tif
   Completed rigid reg on batch in :0.998887300491333s
   Shifted the mov in: 0.30773377418518066s
         After all GPU Batches:Total

/home/ali/packages/s2p-lbm/suite3d/quality_metrics.py:22: RuntimeWarning: divide by zero encountered in divide
  'signal_to_background_ratio' : sig / bg,


         After full batch saving:Total Used: 132.759 GB, Virtual Available: 373.353 GB, Virtual Used: 129.843 GB, Swap Used: 002.916 GB
         Memory at batch 1.  Total Used: 132.760 GB, Virtual Available: 373.352 GB, Virtual Used: 129.844 GB, Swap Used: 002.916 GB
Loading Batch 1 of 54
    Loaded file into shared memory in 2.06 sec


...

Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid st

    Workers completed in 1.27 sec
    Total time: 3.43 sec
   Loaded 1 files, total 0.32 GB
   Batch 53 IO thread joined
         Memory after IO thread joinTotal Used: 133.105 GB, Virtual Available: 373.122 GB, Virtual Used: 130.074 GB, Swap Used: 003.032 GB
         Memory after movie copied from threadTotal Used: 132.434 GB, Virtual Available: 373.793 GB, Virtual Used: 129.402 GB, Swap Used: 003.032 GB
         Memory after thread memory clearedTotal Used: 132.118 GB, Virtual Available: 374.109 GB, Virtual Used: 129.086 GB, Swap Used: 003.032 GB
   Launching IO thread for next batch
         After IO thread launch:Total Used: 132.111 GB, Virtual Available: 374.116 GB, Virtual Used: 129.079 GB, Swap Used: 003.032 GB
      Loaded batch of size (12, 50, 500, 576)
      Loading /mnt/zortex-subjects/CR031/2024-12-06/1/2024-12-06_1_CR031_2P_00001_00055.tif


Exception in thread Thread-306:
Traceback (most recent call last):
  File "/home/ali/anaconda3/envs/suite3d-gpu/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/ali/anaconda3/envs/suite3d-gpu/lib/python3.8/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/home/ali/anaconda3/envs/suite3d-gpu/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ali/packages/s2p-lbm/suite3d/iter_step.py", line 1366, in io_thread_loader
    loaded_mov = jobio.load_data(tifs)
  File "/home/ali/packages/s2p-lbm/suite3d/io/s3dio.py", line 78, in load_data
    mov_list = _dataloader(paths, params, verbose=verbose, debug=debug)
  File "/home/ali/packages/s2p-lbm/suite3d/developer.py", line 56, in wrapper
    return func(*args, **kwargs)
  File "/home/ali/packages/s2p-lbm/suite3d/io/s3dio.py", line 241, in _load_lbm_tifs
    im = load_and_stitch_full_tif_mp(
  File "/h

   Completed rigid reg on batch in :1.9074885845184326s
   Shifted the mov in: 0.2706291675567627s
         After all GPU Batches:Total Used: 133.557 GB, Virtual Available: 372.670 GB, Virtual Used: 130.526 GB, Swap Used: 003.032 GB
      Saving fused, registered file of shape (12, 50, 587, 676) to /mnt/md0/runs/s3d-CR031_2024-12-06_1/registered_fused_data/fused_reg_data0053.npy
         Saved in 0.39 sec
      Computing quality metrics and saving
         After full batch saving:Total Used: 132.671 GB, Virtual Available: 373.556 GB, Virtual Used: 129.639 GB, Swap Used: 003.032 GB
         Memory at batch 54. Total Used: 132.671 GB, Virtual Available: 373.556 GB, Virtual Used: 129.639 GB, Swap Used: 003.032 GB
Loading Batch 54 of 54
   Batch 54 IO thread joined
         Memory after IO thread joinTotal Used: 132.671 GB, Virtual Available: 373.556 GB, Virtual Used: 129.639 GB, Swap Used: 003.032 GB
         Memory after movie copied from threadTotal Used: 131.783 GB, Virtual Available: 

IndexError: tuple index out of range

In [ ]:
mov_full = job.get_registered_movie('registered_fused_data', 'fused')
im_full = mov_full[:,:200].mean(axis=1).compute()

In [ ]:
tfu.show_tif(im_full[0])

In [ ]:
job.params['cell_filt_type'] = 'gaussian'

job.params['voxel_size_um'] = (20, 3.33, 3.33)
job.params['npil_filt_xy_um'] = 70
job.params['cell_filt_xy_um'] = 5
job.params['sdnorm_exp'] = 0.80

params_to_sweep = {
    'cell_filt_xy_um' : (5,10),
    'npil_filt_xy_um' : (40, 70, 100),
    'sdnorm_exp' : (0.8,1.0),
}
job.sweep_corrmap(params_to_sweep, iter_limit=1)

In [ ]:
%%time
corr_map = job.calculate_corr_map()

In [ ]:
res = job.load_corr_map_results()
vmap = res['vmap']

In [ ]:
vmap = res['vmap']
pct50s = n.percentile(vmap, 50, axis=(1,2))
pct99p9s = n.percentile(vmap, 99.9, axis=(1,2))
vmap[0] -= (pct50s[0] - pct50s[1])

In [ ]:
# most important parameter - any value of the corrmap
# above this will be considered a peak for a possible ROI,
# and will be used as a "seed" to grow an ROI around it
# bigger number: fewer ROIs, only bright ones
# smaller number: many ROIs, increasingly worse quality
job.params['peak_thresh'] = 2.9

# optionally, bin the movie in time to speed up detection
# probably a good idea if you have high framerate (>5 Hz?)
job.params['detection_timebin'] = 1 

# when extending an ROI, compare its activity to its neighboring pixels
# in frames where the fluorescence is above this percentile
job.params['percentile'] = 99.5


job.segment_rois()

In [ ]:
# rois_dir_path = job.combine_patches(n.arange(80), job.dirs['rois'], deduplicate=False,
#                                     parent_dir_name='segmentation', info_use_idx=None)

In [ ]:
job.compute_npil_masks(stats_dir = job.dirs['rois'])


In [ ]:

traces = job.extract_and_deconvolve(stats_dir=job.dirs['rois'],iscell=iscell)

In [ ]:
job.export_results('/mnt/zeytin-f1/s3d-results/',result_dir_name='rois')